# Optimized Quantum ML Training on Colab Pro
## Target: 82% → 90% Accuracy with Trainable Quantum Layers

This notebook is optimized for Google Colab Pro with A100 GPU.

In [ ]:
# Cell 1: Check GPU and Request A100 if needed
import subprocess
import sys

# Check GPU
gpu_info = subprocess.check_output("nvidia-smi", shell=True).decode('utf-8')
print(gpu_info)

if 'A100' in gpu_info:
    print("✅ A100 GPU detected - Optimal for quantum simulation!")
elif 'V100' in gpu_info:
    print("✅ V100 GPU detected - Good for quantum simulation")
else:
    print("⚠️ Consider factory resetting runtime to get A100/V100")
    print("Runtime -> Factory reset runtime -> Change runtime type -> GPU: A100")

In [ ]:
# Cell 2: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Verify data exists
import os
data_path = '/content/drive/MyDrive/set'
if os.path.exists(data_path):
    print(f"✅ Data found at {data_path}")
    print(f"   Train: {os.path.exists(data_path + '/train')}")
    print(f"   Test: {os.path.exists(data_path + '/test')}")
else:
    print(f"❌ Data not found at {data_path}")
    print("Please upload your dataset to Google Drive")

In [ ]:
# Cell 3: Install optimized dependencies
!pip install torch torchvision --upgrade --quiet
!pip install pennylane pennylane-lightning-gpu --quiet
!pip install opencv-python scikit-learn tqdm matplotlib --quiet

# Verify installations
import torch
import pennylane as qml

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0)}")
print(f"PennyLane version: {qml.__version__}")

# Test quantum device
try:
    dev = qml.device('lightning.gpu', wires=4)
    print("✅ lightning.gpu quantum device ready")
except:
    print("❌ lightning.gpu not available, using default.qubit")

In [ ]:
# Cell 4: Clone or update repository
repo_path = '/content/Quanvolutional-Neural-Network'

if os.path.exists(repo_path):
    print("Repository exists, pulling latest changes...")
    %cd {repo_path}
    !git pull
else:
    print("Cloning repository...")
    !git clone https://github.com/YOUR_USERNAME/Quanvolutional-Neural-Network.git
    %cd {repo_path}

# List files to verify
!ls -la src/

In [ ]:
# Cell 5: Configure for Colab with optimizations
import os
import sys

# Add src to path
sys.path.append('/content/Quanvolutional-Neural-Network')

# Set environment variables for optimal performance
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
os.environ['PENNYLANE_COMPILE_CACHE'] = '1'  # Enable compilation cache
os.environ['XLA_PYTHON_CLIENT_PREALLOCATE'] = 'false'
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:512'

# Configure for maximum GPU utilization
torch.backends.cudnn.benchmark = True
torch.backends.cuda.matmul.allow_tf32 = True

print("✅ Colab optimizations configured")

In [ ]:
# Cell 6: Prevent disconnection (run in browser console)
from IPython.display import Javascript

display(Javascript('''
function ClickConnect(){
    console.log("Keeping Colab alive...");
    document.querySelector("colab-toolbar-button#connect").click()
}
setInterval(ClickConnect, 60000)
'''))

print("✅ Keep-alive script activated")
print("Note: For longer training, also paste the script in browser console (F12)")

## Quick Performance Benchmark

In [ ]:
# Cell 7: Benchmark quantum simulation performance
import time
from src.trainable_quantum_model import create_enhanced_model

device = torch.device('cuda')
model = create_enhanced_model(circuit_type='data_reuploading').to(device)

# Test batch
batch_sizes = [32, 64, 128, 256]
for bs in batch_sizes:
    try:
        x = torch.randn(bs, 1, 32, 32).to(device)
        
        # Warmup
        _ = model(x)
        torch.cuda.synchronize()
        
        # Time
        start = time.time()
        _ = model(x)
        torch.cuda.synchronize()
        elapsed = time.time() - start
        
        print(f"Batch size {bs}: {elapsed:.3f}s ({bs/elapsed:.1f} samples/sec)")
    except RuntimeError as e:
        print(f"Batch size {bs}: Out of memory")
        torch.cuda.empty_cache()

print("\n✅ Recommended batch size: 128-256 for A100")

## Main Training Loop

In [ ]:
# Cell 8: Main training with enhanced model
from src.enhanced_training import run_enhanced_training
from src.config import NUM_CLASSES

# Configure training
training_config = {
    'circuit_type': 'data_reuploading',  # Best for accuracy
    'num_epochs': 100,
    'target_accuracy': 90.0,
    'checkpoint_dir': '/content/drive/MyDrive/quantum_checkpoints'
}

print("Starting enhanced training...")
print(f"Configuration: {training_config}")
print("="*50)

# Run training
best_val_acc, test_acc = run_enhanced_training(
    circuit_type=training_config['circuit_type'],
    num_epochs=training_config['num_epochs']
)

print("\n" + "="*50)
print(f"Training Complete!")
print(f"Best Validation Accuracy: {best_val_acc:.2f}%")
print(f"Test Accuracy: {test_acc:.2f}%")
print(f"Improvement over baseline: {best_val_acc - 82.0:.2f}%")
print("="*50)

## Experimental Validation

In [ ]:
# Cell 9: Run comprehensive experiments
from experiments.run_experiments import ExperimentalFramework

framework = ExperimentalFramework(
    base_dir='/content/drive/MyDrive/quantum_experiments'
)

# Run specific experiment based on needs
print("Select experiment to run:")
print("1. Baseline comparison (Fixed vs Trainable)")
print("2. Circuit architecture comparison")
print("3. Ablation study")
print("4. Gradient analysis")

# For automated run, uncomment desired experiment:
# results = framework.run_baseline_comparison()
# results = framework.run_circuit_comparison()
# results = framework.run_ablation_study()
# results = framework.run_gradient_analysis()

## Visualization and Analysis

In [ ]:
# Cell 10: Visualize training progress
import matplotlib.pyplot as plt
import json

# Load training history
checkpoint_dir = '/content/drive/MyDrive/quantum_checkpoints'
history_file = f'{checkpoint_dir}/training_history.json'

if os.path.exists(history_file):
    with open(history_file, 'r') as f:
        history = json.load(f)
    
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    # Loss curves
    axes[0].plot(history['train_loss'], label='Train Loss')
    axes[0].plot(history['val_loss'], label='Val Loss')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss')
    axes[0].legend()
    axes[0].set_title('Training Loss')
    
    # Accuracy curves
    axes[1].plot(history['train_acc'], label='Train Acc')
    axes[1].plot(history['val_acc'], label='Val Acc')
    axes[1].axhline(y=82, color='r', linestyle='--', label='Baseline (82%)')
    axes[1].axhline(y=90, color='g', linestyle='--', label='Target (90%)')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Accuracy (%)')
    axes[1].legend()
    axes[1].set_title('Training Accuracy')
    
    plt.tight_layout()
    plt.show()
    
    # Print final stats
    print(f"Final Validation Accuracy: {history['val_acc'][-1]:.2f}%")
    print(f"Best Validation Accuracy: {max(history['val_acc']):.2f}%")
    print(f"Improvement: {max(history['val_acc']) - 82.0:.2f}%")
else:
    print("No training history found. Run training first.")

## Save Final Model

In [ ]:
# Cell 11: Save model for publication
import torch
from datetime import datetime

# Create publication directory
pub_dir = f'/content/drive/MyDrive/quantum_publication_models'
os.makedirs(pub_dir, exist_ok=True)

# Save with metadata
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
model_path = f'{pub_dir}/quantum_model_90pct_{timestamp}.pth'

# Load best model
best_model_path = f'{checkpoint_dir}/best_model.pth'
if os.path.exists(best_model_path):
    checkpoint = torch.load(best_model_path)
    
    # Add metadata
    checkpoint['metadata'] = {
        'baseline_accuracy': 82.0,
        'achieved_accuracy': checkpoint.get('val_acc', 0),
        'improvement': checkpoint.get('val_acc', 0) - 82.0,
        'architecture': 'trainable_quantum_cnn',
        'quantum_device': 'lightning.gpu',
        'training_device': torch.cuda.get_device_name(0),
        'timestamp': timestamp
    }
    
    # Save
    torch.save(checkpoint, model_path)
    print(f"✅ Model saved to {model_path}")
    print(f"   Accuracy: {checkpoint['metadata']['achieved_accuracy']:.2f}%")
    print(f"   Improvement: {checkpoint['metadata']['improvement']:.2f}%")
else:
    print("No best model found. Complete training first.")

## Cleanup and Summary

In [ ]:
# Cell 12: Training summary and next steps
print("="*60)
print("TRAINING SESSION COMPLETE")
print("="*60)
print()
print("Results saved to Google Drive:")
print(f"  - Checkpoints: /content/drive/MyDrive/quantum_checkpoints/")
print(f"  - Experiments: /content/drive/MyDrive/quantum_experiments/")
print(f"  - Final models: /content/drive/MyDrive/quantum_publication_models/")
print()
print("Next steps:")
print("1. Download results to Mac for analysis")
print("2. Generate publication figures")
print("3. Run statistical significance tests")
print("4. Prepare manuscript with results")
print()
print("To sync results to Mac:")
print("  rsync -av /path/to/drive/quantum_* ~/Desktop/results/")
print("="*60)